In [ ]:
import os
import sys
import logging
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

from dotenv import load_dotenv

load_dotenv()

# API 키 설정
api_key = os.getenv("OPENAI_API_KEY")

if not api_key:
    logging.error("API 키를 설정해주세요.")

# LLM 초기화
llm = ChatOpenAI(model="gpt-4o-mini", temperature=1.0)

/var/folders/dn/cz4fv22x5s7bp8_g1_qj6k_w0000gn/T/ipykernel_78984/944493611.py:19: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(model="gpt-4o-mini", temperature=1.0)


In [14]:
# === 문서 로드 및 인덱스 생성 (RAG) ===
# 문서 폴더 내에는 아래 데이터가 포함되어 있다고 가정.
# 1. 합격 자소서 데이터, 2. 합격자 면접 데이터, 3. 업계 동향 데이터, 4. 업계 구인 데이터

# 문서 로드를 위한 패키지
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.core import Document
from llama_index.core import GPTVectorStoreIndex

# 문서 로드 및 인덱스 생성
data_path = os.getenv("DATA_PATH")
documents = SimpleDirectoryReader(data_path).load_data()
if not data_path:
    logging.error("데이터 경로를 설정해주세요.")

index = GPTVectorStoreIndex.from_documents(documents)
query_engine = index.as_query_engine()

In [15]:
# 참고 자료 검색: 자소서 분석 및 피드백에 유용한 자료를 외부 문서에서 추출
reference_query = (
    "합격 자소서, 합격자 면접, 업계 동향, 구인 데이터를 참고하여, "
    "자소서 작성 피드백에 유용한 핵심 인사이트를 간략히 요약해줘."
)
reference_result = query_engine.query(reference_query)

# 지원 기업 관련 산업 뉴스 데이터 추출 (지원 기업 및 관련 키워드 활용)
company = "A기업"
industry_news_query = (
    f"{company} 관련 산업 뉴스, 최신 동향, 전략 및 이슈"
)
industry_news_result = query_engine.query(industry_news_query)


# 두 결과를 결합하여 참고 자료로 활용
combined_reference = f"{reference_result}\n\n[산업 뉴스 참고]:\n{industry_news_result}"


In [16]:
## 여기부터 실행

# === 자소서 분석 체인 프롬프트 템플릿 ===
init_prompt = PromptTemplate(
    input_variables=["참고자료"],
    template="""
    이전 대화 내용을 무시하고, 아래에 제공된 정보만을 기반으로 분석을 수행해 주세요.
    이후에 입력하는 내용을 기반으로 자소서 분석, 조언, 피드백을 제공해주세요.
    --------------------------------------
    {참고자료} 
    --------------------------------------
    """
)

In [26]:
# === 수정된 분석 체인 프롬프트 템플릿 (회사, 직무 정보 포함 및 적합성 체크) ===
analysis_prompt = PromptTemplate(
    input_variables=["자소서내용", "참고자료", "회사", "직무"],
    template="""
사용자가 입력한 자소서 내용은 다음과 같습니다:
--------------------------------------
{자소서내용}
--------------------------------------
아래의 참고 자료는 합격 자소서, 면접 데이터, 업계 동향, 구인 데이터 및 지원 기업 관련 산업 뉴스에서 추출된 인사이트입니다:
--------------------------------------
{참고자료}
--------------------------------------
또한, 사용자가 지원하는 회사는 "{회사}"이며 희망 직무는 "{직무}"입니다.
위 정보를 종합하여, 다음 사항들을 분석해줘:
1. 자소서에 나타난 강점과 약점을 각각 3가지씩 구체적으로 제시
2. 자소서 내용이 지원하는 회사와 희망 직무에 얼마나 부합하는지 평가하고, 개선이 필요한 부분(예: 지원 동기, 관련 경험, 역량 표현 등)을 구체적으로 지적
3. 최종적으로, 해당 자소서의 지원 적합성 점수를 0부터 10까지의 수치로 산출해줘.
예시 출력 형식:
- 강점: [예: 문제 해결 능력, 팀워크 등]
- 약점: [예: 경험 부족, 구체성 부족 등]
- 적합성 평가: [예: 지원 동기 보완 필요 등]
- 지원 적합성 점수: [예 : 5/10]
"""
)

In [27]:
# === 조언 체인 프롬프트 템플릿 (회사, 직무 정보 추가) ===
advice_prompt = PromptTemplate(
    input_variables=["분석결과", "회사", "직무"],
    template="""
아래 분석 결과와 지원 정보(지원 회사 및 직무)를 참고하여, 자소서 개선 및 보완을 위한 구체적인 조언을 작성해줘.
분석 결과:
--------------------------------------
{분석결과}
--------------------------------------
지원 정보:
- 지원하는 회사: {회사}
- 지원하는 직무: {직무}

조언은 실제 자소서에 적용 가능한 수정 방안과 개선 방향을 상세하게 제시하고, 지원 회사와 직무에 맞는 어필 포인트를 반영해줘.
그리고 분석결과에서 출력된 점수도 다시 출력해줘.
"""
)

In [28]:
# === 자기소개서 예시 출력 프롬프트 템플릿 (회사, 직무 정보 포함) ===
example_prompt = PromptTemplate(
    input_variables=["조언결과", "회사", "직무", "자기소개서"],
    template="""
아래 조언 결과를 바탕으로, 실제 지원자가 제출할 수 있는 자기소개서 예시를 작성해줘.
자기소개서는 다음과 같은 구조를 갖추어야 합니다:
1. **지원동기 및 비전:** 지원 동기, 본인의 비전, 그리고 지원 회사({회사}) 및 직무({직무})와의 시너지를 명확히 서술
2. **경험 및 역량:** 학습, 동아리/프로젝트 경험, 보유 기술 및 역량을 구체적으로 기술
3. **강점 및 보완 사항:** 분석된 강점을 부각시키고, 약점을 개선하기 위한 구체적인 실행 계획 제시
4. **향후 계획:** 입사 후 성장 방향과 목표

각 항목별로 소제목을 붙여 기존 자기소개서에 조언결과를 적용한 결과를 예시로 보여줘.
예시 자기소개서는 기존 자기소개서의 분량과 비슷하게 작성해줘.
아래 조언 결과:
--------------------------------------
{조언결과}
--------------------------------------
자기소개서 예시:
{자기소개서}
"""
)

In [29]:
# 체인 생성: 분석 체인, 조언 체인, 예시 출력 체인
init_chain = LLMChain(llm=llm, prompt=init_prompt)
analysis_chain = LLMChain(llm=llm, prompt=analysis_prompt)
advice_chain = LLMChain(llm=llm, prompt=advice_prompt)
example_chain = LLMChain(llm=llm, prompt=example_prompt)

In [30]:
# === 사용자의 입력 ===
user_self_intro = """
**지원 동기 및 비전: 최고의 사용자 경험을 만드는 소프트웨어 엔지니어로서의 꿈**  
현재 IT 산업은 빠르게 변화하고 있으며, 사용자들은 단순히 기능적 요구를 충족하는 것을 넘어 직관적이고 매끄러운 인터랙션을 기대하고 있습니다. 특히 가비아의 비전인 “advanced industries”에 깊이 공감하며, 고객이 다가갈 수 있는 모든 디지털 접점을 최적의 경험으로 이끄는 것에 기여하고 싶습니다. 전자공학을 전공하면서 하드웨어와 소프트웨어의 통합이 주는 가능성을 직간접적으로 경험하였고, 특히 사용자 중심의 개발이 얼마나 중요한지 깨닫게 되었습니다. 이를 바탕으로, 가비아의 소프트웨어 엔지니어로서 최신 기술을 활용한 최상의 사용자 경험을 제공하고 싶습니다.

**경험 및 역량: 다양한 프로젝트로 다져온 소프트웨어 개발 실력**  
저는 웹 개발에 집중하며 React, TypeScript 등을 활용한 여러 프로젝트를 진행했습니다. 이 중 <스마트 홈 관리 시스템> 프로젝트에서 사용자 피드백을 반영하여 UI/UX를 개선했으며, 이로 인해 사용자 만족도가 30% 향상되었습니다. 특히 Next.js를 활용하여 서버 사이드 렌더링(SSR)을 구현해 페이지 로딩 속도를 40% 줄이고 사용자 이탈률을 크게 낮춘 경험이 있습니다.

또한, 대학 시절 동아리에서 진행한 프로젝트를 통해 Atomic Design 패턴을 적용해 UI 컴포넌트를 설계했으며, Storybook을 활용하여 독립적인 컴포넌트 개발 및 테스트를 진행하는 방법을 익혔습니다. 이러한 경험은 재사용성과 유지보수성을 고려한 개발 능력을 키우는데 큰 도움이 되었습니다.

**강점 및 보완 사항: 협업 능력과 성과 중심의 접근**  
저의 가장 큰 강점은 사용자 피드백을 적극적으로 반영하여 개선하는 능력입니다. 특히 협업 과정에서 열린 마음으로 다른 개발자의 의견을 경청하고, 최적의 해결책을 도출하기 위해 노력합니다. 한편, 제 약점은 때로 지나친 완벽주의가 프로젝트 일정에 영향을 미칠 수 있다는 점입니다. 이를 보완하기 위해, 개인적으로 ‘고-로우’ 목표 설정 전략을 수립하고, 우선 실행할 기능과 개선할 사항을 차별화하여 보다 유연하게 프로젝트를 관리하고 있습니다.

**향후 계획: 가비아와 함께 성장하며 차별화된 사용자 경험 제공**  
가비아에서 제 목표는 프론트엔드 개발자로서 더욱 사용자 친화적이고 성능 최적화된 애플리케이션을 개발하는 것입니다. 특히, 고객의 요구를 실시간으로 반영할 수 있는 시스템 개선이나 새로운 기능 추가를 통해 사용자들이 더욱 편리하게 사용할 수 있도록 기여하고 싶습니다. 입사 후에는 팀원들과의 원활한 소통을 통해 협력적인 개발 환경을 만들어가며, 지속적인 자기 개발을 통해 최고의 소프트웨어 엔지니어로 성장하는 것이 제 비전입니다.

가비아에서의 기회를 통해 더 나은 사용자 경험을 창출하고, 함께 성장하는 모습을 보여드리겠습니다.
"""

# 추가 입력: 지원하는 회사와 직무
company = "가비아"
job_position = "소프트웨어 엔지니어"

In [31]:
# 0. 초기화 체인: 이전 프롬프트의 내용을 초기화하고, 새로운 참고 자료를 입력
init_result = init_chain.run(
    참고자료=combined_reference
)

In [32]:
# 1. 분석 체인 실행: 자소서 내용과 참고 자료, 지원 정보를 기반으로 강점/약점 및 적합성 평가
analysis_result = analysis_chain.run(
    자소서내용=user_self_intro,
    참고자료=combined_reference,
    회사=company,
    직무=job_position
)
print("=== 분석 결과 ===")
print(analysis_result)

=== 분석 결과 ===
1. **강점**:
   - **사용자 중심 개발 경험**: 자소서에서 여러 프로젝트를 통해 사용자 피드백을 반영하여 UI/UX를 개선한 경험을 강조하고 있어, 사용자의 요구를 잘 이해하고 있다는 점에서 강점이 됩니다.
   - **실질적인 성과**: 스마트 홈 관리 시스템 프로젝트의 단계에서 사용자 만족도를 30% 향상시킨 점과 페이지 로딩 속도를 40% 줄인 것은 실질적인 성과로 보여줍니다.
   - **협업 능력**: 다른 개발자의 의견을 경청하고 열린 마음으로 접근하는 자세가 언급되어, 팀 프로젝트에서의 협업 능력이 강점으로 부각됩니다.

2. **약점**:
   - **완벽주의 경향**: 지나치게 완벽주의적이라는 점이 프로젝트 일정에 영향을 미칠 수 있다고 언급하면서, 이를 보완하기 위한 전략을 세우고 있다고 하지만, 약점으로 남아있습니다.
   - **구체성 부족**: 여러 프로젝트에서 어떤 구체적인 기술적 문제를 해결했던 경험이 좀 더 명확하게 서술될 필요가 있으며, 보완된 경험이 필요합니다.
   - **가비아에 대한 구체적 이해 부족**: 가비아의 비전이나 사업 모델에 대한 구체적인 내용이 조금 부족하여, 지원 동기가 회사에 대한 이해를 충분히 반영하지 못하고 있는 것으로 보입니다.

3. **적합성 평가**: 
   - 자소서의 전반적인 내용은 소프트웨어 엔지니어로서의 경험과 기술을 잘 표현하고 있지만, 가비아에 대한 깊은 이해와 연관된 경험을 더 구체적으로 서술하는 것이 필요합니다. 사용자 경험에 대한 강조와 실질적인 기술적 성취는 긍정적이지만, 약간의 보완이 필요합니다. 따라서, 전반적으로 적합성 점수는 중간 이상 수치를 부여할 수 있습니다.

- **지원 적합성 점수**: 7/10


In [33]:
# 2. 조언 체인 실행: 분석 결과와 지원 정보를 기반으로 개선 조언 생성
advice_result = advice_chain.run(
    분석결과=analysis_result,
    회사=company,
    직무=job_position
)
print("\n=== 조언 결과 ===")
print(advice_result)


=== 조언 결과 ===
자소서 개선 및 보완을 위한 구체적인 조언을 아래와 같이 제시합니다. 가비아라는 회사와 소프트웨어 엔지니어라는 직무에 맞춘 어필 포인트와 수정 방안을 함께 고려했습니다.

### 1. 사용자 중심 개발 경험 강조
- **구체화**: 현재 지닌 사용자 중심 개발 경험을 더욱 구체적으로 서술하세요. 예를 들어 “사용자 피드백을 반영하여 UI/UX를 개선하며, 구체적인 변화 내용을 기술적으로 설명합니다. ‘어떤 피드백이 있었는지’, ‘그걸 반영하며 어떤 기술을 사용했는지’, ‘결과적으로 어떤 변화가 있었는지’ 등의 세부사항을 추가하여 더 설득력 있게 어필할 수 있습니다.
  
### 2. 실질적인 성과 상세화
- **수치로 표현**: 성과가 인상적이지만, 이를 더욱 강화하려면 ‘어떤 사용자 조사 도구’를 사용했는지와 같은 접근 방법, ‘페이지 로딩 속도 감소에 기여한 기술 스택’을 구체적으로 언급하세요. 예를 들어, “React.js를 통한 최적화”와 같은 기술적 부분을 추가하여 귀하의 전문성을 강조할 수 있습니다.

### 3. 완벽주의 경향 보완 전략
- **성 성장 이야기로 변환**: 완벽주의가 문제라면, 이를 극복하고 있는 과정이나 그로 인해 배운 긍정적인 경험을 구체화하세요. 예를 들어, “이 부분이 어려웠으나, 일정 지키기를 위해 팀원과 조율하며 결과적으로 XX 프로젝트를 성공적으로 완료할 수 있었다” 같은 긍정적인 결과를 서술하면 좋습니다.

### 4. 구체적인 기술적 문제 해결 경험 추가
- **프로젝트 사례**: 각 프로젝트에서 해결했던 구체적인 기술적 문제에 대한 개인적인 역할을 서술하세요. “이 문제를 해결하기 위해 XXX 기술을 사용했으며, 그 결과 YYY 효과를 얻었다”고 명확하게 연계 지어 주시면 좋습니다.

### 5. 가비아에 대한 이해 심화
- **회사 연구 및 맞춤형 지원 동기**: 가비아의 비전, 사업 모델(예: 호스팅, 클라우드 서비스 등) 및 최근의 트렌드(예: AI 활용, 소프트웨어 솔루션)에 대해

In [34]:
# 3. 예시 출력 체인 실행: 조언 결과와 지원 정보를 반영한 자기소개서 예시 생성
example_self_intro = example_chain.run(
    조언결과=advice_result,
    회사=company,
    직무=job_position,
    자기소개서=user_self_intro
)
print("\n=== 자기소개서 예시 ===")
print(example_self_intro)


=== 자기소개서 예시 ===
**지원 동기 및 비전: 최고의 사용자 경험을 만드는 소프트웨어 엔지니어로서의 꿈**  
IT 산업이 급속히 발전하는 시대에, 사용자들은 단순한 기능적 요구를 넘어서 직관적이고 매끄러운 인터랙션을 기대하고 있습니다. 특히 가비아의 비전인 “advanced industries”에 깊이 공감하며, 고객이 다가갈 수 있는 모든 디지털 접점을 최적의 경험으로 이끄는 데 기여하고 싶습니다. 전자공학을 전공하며 하드웨어와 소프트웨어의 통합가능성을 체험하였고, 특히 사용자 중심의 개발이 얼마나 중요한지를 깨닫게 되었습니다. 이를 바탕으로, 가비아의 소프트웨어 엔지니어로서 최신 기술을 활용하여 최상의 사용자 경험을 제공하고 싶습니다.

**경험 및 역량: 다양한 프로젝트로 다져온 소프트웨어 개발 실력**  
저는 웹 개발에 집중하여 React.js와 TypeScript 등을 활용한 여러 프로젝트를 진행했습니다. 특히 <스마트 홈 관리 시스템> 프로젝트에서 사용자 피드백을 반영하여 UI/UX를 개선했습니다. 초기 설문조사에서 수집한 피드백을 통해 30%의 사용자 만족도 향상을 이루었고, Next.js를 사용하여 서버 사이드 렌더링(SSR)을 구현해 페이지 로딩 속도를 40% 감소시킨 경험이 있습니다. 이를 통해 사용자 이탈률을 큰 폭으로 낮추는 효과도 있었습니다.

또한, 대학 시절 동아리에서 진행한 프로젝트에서 Atomic Design 패턴을 적용해 UI 컴포넌트를 설계하였고, Storybook을 활용하여 독립적인 컴포넌트를 개발 및 테스트하는 과정을 통해 재사용성과 유지보수성을 고려한 개발 능력을 키웠습니다. 이와 같은 경험들은 가비아의 소프트웨어 엔지니어로서 실질적인 성과를 창출하는 기반이 될 것입니다.

**강점 및 보완 사항: 협업 능력과 성과 중심의 접근**  
저의 가장 큰 강점은 사용자 피드백을 적극 반영하여 개선하는 능력입니다. 협업 과정에서도 열린 마음으로 타 개발자의 의견을 경청하고 최적의 해결책을 찾기 위해 노력합니다. 반